# データを操作する

データは、機械学習モデルが構築される基盤です。クラウドで一元的にデータを管理し、複数のワークステーションとコンピューティング ターゲットで実験を実行してモデルをトレーニングしているデータ サイエンティストのチームがアクセスできるようにすることは、プロフェッショナルなデータ サイエンス ソリューションでは重要です。

このノートブックでは、データを操作するための 2 つの Azure Machine Learning オブジェクト、*データストア*と*データセット*について学びます。

## ワークスペースに接続する

作業を開始するには、ワークスペースに接続します。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存された構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## データストアを操作する

Azure ML では、*データストア*は、Azure Storage Blob コンテナーのようなストレージ場所への参照です。あらゆるワークスペースには既定のデータストアがあります。通常は、ワークスペースで作成された Azure Storage Blob コンテナーです。異なる場所に格納されているデータを使用する必要がある場合は、ワークスペースにカスタム データストアを追加して、既定に設定することができます。

### データストアを表示する

次のコードを実行してワークスペースでデータストアを判定します:

In [ ]:
# 既定のデータストアを取得する
default_ds = ws.get_default_datastore()

# すべてのデータストアを列挙し、どちらが既定かを示す
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

また、[Azure Machine Learning Studio](https://ml.azure.com) のワークスペースに関する「**データストア**」ページでワークスペースのデータストアを表示して管理することもできます。

### データストアにデータをアップロードする

利用可能なデータストアを特定したので、実際に実験スクリプトが実行されている場所に関係なく、ワークスペースで実行中の実験にアクセスできるように、ローカル ファイル システムからファイルをアップロードできます。

In [ ]:
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

## データセットを操作する

Azure Machine Learning は*データセット*というかたちでデータの抽象化を提供します。データセットは、実験で使用したい特定のデータのセットへの参照で、バージョン管理されています。データセットは*表形式*または*ファイル*ベースのいずれかになります。

### 表形式データセットを作成する

データストアにアップロードした糖尿病データからデータセットを作成し、最初の 20 件のレコードを表示してみましょう。この場合、データは CSV ファイル内の構造化された形式なので、*表形式*のデータセットを使用します。

In [ ]:
from azureml.core import Dataset

# 既定のデータストアを取得する
default_ds = ws.get_default_datastore()

#データストア上のパスから表形式のデータセットを作成する (しばらく時間がかかる場合があります)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 最初の 20 行を Pandas データフレームとして表示する
tab_data_set.take(20).to_pandas_dataframe()

上のコードでわかるように、表形式のデータセットを Pandas データフレームに変換するのは簡単で、一般的な Python の手法を使用してデータを操作できます。

### ファイル データセットを作成する

作成したデータセットは、データセット定義に含まれる構造化ファイル内のすべてのデータを含むデータフレームとして読み取ることができる*表形式*のデータセットです。これは表形式のデータに適していますが、機械学習のシナリオによっては、非構造化データの操作が必要となる場合があります。または、単に自分のコード内のファイルからデータの読み取り処理を行うことが必要となる場合もあります。これを実現するには、*ファイル* データセットを使用して、ファイルのデータを読み取るために使用できる仮想マウント ポイント内のファイル パスのリストを作成します。

In [ ]:
#データストア上のパスからファイル データセットを作成する (しばらく時間がかかる場合があります)
file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))

# データセット内のファイルを取得する
for file_path in file_data_set.to_path():
    print(file_path)

### データセットを登録する

これで、糖尿病データを参照するデータセットを作成したので、それらを登録して、ワークスペースで実行されている実験に簡単にアクセスできるようにすることができます。

表形式のデータセットを**糖尿病データセット**、ファイル データセットを**糖尿病ファイル**として登録します。

In [ ]:
# 表形式のデータセットを登録する
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                        name='diabetes dataset',
                                        description='diabetes data',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

# ファイル データセットを登録する
try:
    file_data_set = file_data_set.register(workspace=ws,
                                            name='diabetes file dataset',
                                            description='diabetes files',
                                            tags = {'format':'CSV'},
                                            create_new_version=True)
except Exception as ex:
    print(ex)

print('Datasets registered')

[Azure Machine Learning Studio](https://ml.azure.com) のワークスペースに関する「**データセット**」ページでデータセットを表示して管理できます。ワークスペース オブジェクトからもデータセットのリストを取得できます。

In [ ]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

データセットをバージョン管理できるため、以前の定義に依存する既存の実験やパイプラインを壊すことなくデータセットを再定義できます。既定では、名前付きデータセットの最新バージョンが返されますが、次のようにバージョン番号を指定することで、特定のバージョンのデータセットを取得できます。

```python
dataset_v1 = Dataset.get_by_name(ws, 'diabetes dataset', version = 1)
```


### 表形式データセットからモデルをトレーニングする

データセットができたので、そこからモデルのトレーニングを開始する準備が整いました。データセットは、スクリプトの実行に使用される Estimator で、*入力*としてスクリプトに渡すことができます。

次の 2 つのコード セルを実行して作成します。

1.**diabetes_training_from_tab_dataset** という名前のフォルダー
2.引数として渡される表形式のデータセットを使用して分類モデルをトレーニングするスクリプト。

In [ ]:
import os

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_training_from_tab_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import os
import argparse
from azureml.core import Run, Dataset
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# スクリプト引数を取得する (正規化率とトレーニング データセット ID)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# 正規化ハイパーパラメーターを設定する (スクリプトに引数として渡される)
reg = args.reg_rate

# 実験実行コンテキストを取得する
run = Run.get_context()

# トレーニング データセットを取得する
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# 特徴とラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# ロジスティック回帰モデルをトレーニングする
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

> **注**: スクリプトでは、データセットはパラメーター (または引数) として渡されます。表形式のデータセットの場合、この引数には登録済みのデータセットの ID が含まれます。このため、スクリプトにコードを書き込んで実験のワークスペースを実行コンテキストから取得した後、以下のような ID を利用してデータセットを取得できます。
>
> ```
> run = Run.get_context()
> ws = run.experiment.workspace
> dataset = Dataset.get_by_id(ws, id=args.training_dataset_id)
> diabetes = dataset.to_pandas_dataframe()
> ```
>
> ただし、Azure Machine Learning 実行では、名前の付いたデータセットを参照して、これを実験の **input_datasets** コレクションに追加する引数が自動的に識別されます。このため、「フレンドリ名」を指定して、このコレクションからデータセットを取得することもできます (後述しますが、この名前は実験のスクリプト実行構成の引数定義で指定されます)。このアプローチは上記のスクリプトで使用されています。

これでスクリプトを実験として実行し、スクリプトで読み取られるトレーニング データセットの引数を定義できます。

> **注**: **Dataset** クラスは、**azureml-dataprep** パッケージの一部のコンポーネントに依存しているため、トレーニング実験の環境にこのパッケージを含める必要があります。**azureml-dataprep** パッケージは、**azure-defaults** パッケージに含まれています。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.widgets import RunDetails


# 実験用の Python 環境を作成する（.yml ファイルから）
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# トレーニング データセットを取得する
diabetes_ds = ws.datasets.get("diabetes dataset")

# スクリプト構成を作成する
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='diabetes_training.py',
                              arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                           '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                              environment=env) 

# 実験を送信する
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

> **"注:****--Input-data** 引数は*名前の付いた入力*としてデータセットを渡します。この中には、スクリプトが実験実行の **input_datasets** コレクションから読み取るために使用する、データセットの*フレンドリ名*が含まれます。**--Input-data** 引数の文字列値は実際には登録済みのデータセットの ID です。  代替アプローチとして、`diabetes_ds.id` を渡すこともできます。この場合、スクリプトはスクリプト引数からデータセット ID にアクセスし、これを使用してデータセットをワークスペースから取得できますが、**input_datasets** コレクションからは取得できません。

初めて実験を実行すると、Python 環境のセットアップに時間がかかる場合があります。以降の実行はより高速になります。

実験が完了したら、ウィジェットで、**azureml-logs/70_driver_log.txt** 出力ログと実行によって生成されたメトリックを表示します。

### トレーニングされたモデルを登録する

他のトレーニング実験と同様、Azure Machine Learning ワークスペースではトレーニングされたモデルを取得して登録することができます。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Tabular dataset'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

### ファイル データセットからモデルをトレーニングする

*表形式*データセットでトレーニング データを使用してモデルをトレーニングする方法を見てきましたが、*ファイル* データセットについてはどうでしょうか。

ファイル データセットを使用する場合、スクリプトに渡されるデータセット引数は、ファイル パスを含むマウント ポイントを表します。これらのファイルからデータを読み取る方法は、ファイル内のデータの種類と、そのデータを使用して何を行うかによって異なります。糖尿病 CSV ファイルの場合、Python **glob** モジュールを使用して、データセットによって定義された仮想マウント ポイント内のファイルのリストを作成し、それらをすべて単一のデータフレームに連結された Pandas データフレームに読み込むことができます。

次の 2 つのコード セルを実行して作成します。

1.**diabetes_training_from_file_dataset** という名前のフォルダー
2.*入力*として渡されるファイル データセットを使用して分類モデルをトレーニングするスクリプト。

In [ ]:
import os

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_training_from_file_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import os
import argparse
from azureml.core import Dataset, Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import glob

# スクリプト引数を取得する (正規化率とファイル データセット マウント ポイント)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument('--input-data', type=str, dest='dataset_folder', help='data mount point')
args = parser.parse_args()

# 正規化ハイパーパラメーターを設定する (スクリプトに引数として渡される)
reg = args.reg_rate

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データセットを読み込む
print("Loading Data...")
data_path = run.input_datasets['training_files'] # Get the training data path from the input
# (ハードコードされた覚えやすい名前を利用したくない場合は、args.data_folder を使用することも可能)

# ファイルを読み取る
all_files = glob.glob(data_path + "/*.csv")
diabetes = pd.concat((pd.read_csv(f) for f in all_files), sort=False)

# 特徴とラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# ロジスティック回帰モデルをトレーニングする
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

表形式のデータセットと同様、ファイル データセットもフレンドリ名を使用して **input_datasets** コレクションから取得できます。また、スクリプト引数から取得することも可能です。この場合、ファイル データセットにはファイルへのマウント パスが含まれます (表形式のデータセットで渡されるデータセット ID とは異なります)。

次に、データセットをスクリプトに渡す方法を変更する必要があります。スクリプトがファイルを読み取るパスを定義しなくてはなりません。これを行うには、**as_download** または **as_mount** メソッドのいずれかを使用できます。**as_download** を使用すると、ファイル データセットのファイルは、スクリプトを実行しているコンピューティングの一時的な場所にダウンロードされます。一方、**as_mount** は、ファイルを直接、データストアからストリームできるマウント ポイントを作成します。

アクセス メソッドを **as_named_input** メソッドと組み合わせ、実験実行で **input_datasets** コレクションにデータセットを含めることができます (引数を `diabetes_ds.as_mount()` に設定するなどしてこれを省略すると、スクリプトはスクリプト引数からデータセット マウント ポイントにアクセスできますが、**input_datasets** コレクションからはアクセスできません)。

In [ ]:
from azureml.core import Experiment
from azureml.widgets import RunDetails


# トレーニング データセットを取得する
diabetes_ds = ws.datasets.get("diabetes file dataset")

# スクリプト構成を作成する
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_named_input('training_files').as_download()], # Reference to dataset location
                                environment=env) # Use the environment created previously

# 実験を送信する
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

実験が完了したら、ウィジェットで **azureml-logs/70_driver_log.txt** 出力ログを表示し、ファイル データセットのファイルが一時的なフォルダーにダウンロードされており、スクリプトがファイルを読み取れることを確認します。

### トレーニングされたモデルを登録する

もう一度、実験によってトレーニングされたモデルを登録できます。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'File dataset'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳細情報**: データセットを使用するトレーニングの詳細については、Azure ML ドキュメントの[データセットを使用するトレーニング](https://docs.microsoft.com/azure/machine-learning/how-to-train-with-datasets)を参照してください。